In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# CIFAR10 데이터 셋 불러오기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root="../data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.CIFAR10(root="../data",train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
# GPU 연산 체크하기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is avaliable")

In [ ]:
# AlexNet 구축하기

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 192, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(192, 384, 3, padding=1), nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256*3*3, 1024), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256*3*3)
        x = self.classifier(x)
        return x

In [ ]:
# 손실 함수 및 최적화 방법 정의하기
# GPU 연산을 위해 모델을 불러올 때 .to(device)를 반드시 붙여야한다.
criterion = nn.CrossEntropyLoss()
alexnet = AlexNet().to(device)
optimizer = optim.Adam(alexnet.parameters(), lr=1e-3)

In [ ]:
# AlexNet 모델 학습하기
loss_ = []
n = len(trainloader)
for epoch in range(50):
    running_loss = 0.0
    for data in trainloader:
        inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터
        optimizer.zero_grad()
        outputs = alexnet(inputs) # 예측값 산출
        loss = criterion(outputs, labels) # 손실 함수 계산
        loss.backward() # 손실 함수 기준으로 역전파 선언
        optimizer.step() # 가중치 최적화
        running_loss += loss.item()
    
    loss_.append(running_loss / n)
    print("[%d] loss : %.3f"%(epoch+1, running_loss / len(trainloader)))

In [ ]:
# 학습 손실 함수 그래프 그리기
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
# 파이토치 모델 저장 및 불러오기
PATH = "./models/cifar_alexnet.pth"
torch.save(alexnet.state_dict(), PATH)

In [ ]:
alexnet = AlexNet().to(device) # 모델 저장은 모델 전체를 저장하는 것이 아닌 모델 파라미터를 저장하는 것이다.
# 즉, 모델을 불러올 떄 모델이 선행적으로 선언되어 있어야 한다.
alexnet.load_state_dict(torch.load(PATH)) # 모델의 파라미터를 불러와서 모델에 주입한다.

In [ ]:
# 평가하기
correct = 0
total = 0
with torch.no_grad():
    alexnet.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0) # 개수 누적
        correct += (predicted == labels).sum().item() # 누적 (맞으면 1, 틀리며 0으로 합산)

print("Test accuracy: %.2f %%"%(100*correct / total))

In [ ]:
# Residual Block 구축하기
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size = 3, stride=stride, padding=1, bias=False), nn.BatchNorm2d(self.out_channels), nn.ReLU(),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False), nn.BatchNorm2d(self.out_channels)
        )
        if self.stride != 1 or self.in_channels != self.out_channels:
                self.downsample = nn.Sequential(
                     nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(self.out_channels)
                )

    def forward(self, x):
        out = self.conv_block(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            x = self.downsample(x)
        out = F.relu(x + out)
        return out

In [ ]:
# ResNet 모델 구축하기
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self._make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.gap = nn.AvgPool2d(4) # 4 필터 사이즈
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1)
        layers = []
        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels
        return nn.Sequential(*layers) # *리스트는 리스트의 길이에 상관없이 모든 성분을 별도로 nn.sequential에 전달하는 역할을 한다.
    
    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
def modeltype(model):
    if model == "resnet18":
        return ResNet([2, 2, 2, 2])
    elif model == "resnet34":
        return ResNet([3, 4, 6, 3])


In [ ]:
resnet = modeltype("resnet18").to(device)
print(resnet)
PATH="./models/cifar_resnet.pth" # 모델 저장 경로

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=1e-3)

In [ ]:
loss_ = []
n = len(trainloader)

for epoch in range(10):

    running_loss = 0.0
    for data in trainloader:

        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    loss_.append(running_loss / n)
    print("[%d] loss : %.3f" %(epoch + 1, running_loss / n))

In [ ]:
# 평가하기
correct = 0
total = 0
with torch.no_grad():
    resnet.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) # 개수 누적
        correct += (predicted == labels).sum().item() # 누적 (맞으면 1, 틀리며 0으로 합산)

print("Test accuracy: %.2f %%"%(100*correct / total))